## Filter
One of the most powerful parameters when defining a search is the ability to filter the results.  While a Query is more of an unstructured approach to search for text within documents, a Filter provides a well-defined structured approach when we know which properties we want to restrict our matching documents.

The following examples will be using some of the popular properties as outlined within the "Properties & Metadata" notebook.  In addition, the selection of these properties was determined based on the discovery techniques used such as metadata and debugging.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.19.0 Refinitiv.Data.Content, 1.0.0-beta3

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

24/01/2022 11:56:17 AM:DesktopSession. (State: DesktopSession is Pending)
24/01/2022 11:56:17 AM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
24/01/2022 11:56:17 AM:DesktopSession. (State: DesktopSession is Opened)


#### Example - Query vs Filter
As you become more familiar with Search, you will realize there are a number of different paths you can take to arrive at the same destination.

In [5]:
var response = Search.Definition(Search.View.Instruments).Query("US046353AQ14")
                                                         .Select("DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID")
                                                         .GetData();
response.Data.Table

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [6]:
// Retrieve the document(s) for the same ISIN using a Filter
response = Search.Definition(Search.View.Instruments).Filter("ISIN eq 'US046353AQ14'")
                                                     .Select("DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID")
                                                     .GetData();
response.Data.Table

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


In [7]:
// If you are provided a code but are not sure if the code is an ISIN, SEDOL, CUSIP etc, 
// try using the 'CodeResolutionBase' property.
// Note: I'm using the CUSIP for the same instrument within the Filter
response = Search.Definition(Search.View.Instruments).Filter("CodeResolutionBase eq '046353AQ1'")
                                                     .Select("DocumentTitle, RIC, ISIN, CUSIP, SEDOL, PermID")
                                                     .GetData();
response.Data.Table

"AstraZeneca PLC, Plain Vanilla Fixed Coupon Bond, TRACE Eligible AZN 2.375 12-Jun-2022",046353AQ1=,US046353AQ14,046353AQ1,BF4SNM7,46645702869


#### Example - Apple Equities

In [8]:
// Within the view (EquityQuotes), select all documents where the term 'AAPL' is within the ticker symbol.  For each, 
// ensure the type of asset is an 'equity' and that each asset is in an active state.  I chose to filter out the 
// AssetState of 'DC' (De-activated) as my criteria.
//
// Note: An alternative to "AssetType eq 'equity'" could be to use: "RCSAssetCategoryLeaf eq 'Ordinary Share'"
var filter = "AssetType eq 'equity' and AssetState ne 'DC' and TickerSymbol eq 'AAPL'";
var select = "RIC, CompositeDescriptiveName, ExchangeCode, ExchangeName, AssetType";

response = Search.Definition(Search.View.EquityQuotes).Filter(filter)
                                                      .Select(select)
                                                      .GetData();
response.Data.Table

AAPL.O,Apple Ord Shs APPLE ORD,NSQ,NASDAQ Global Select Consolidated,[EQUITY]
AAPLEUR.xbo,Apple Ord Shs APPLE ORD APPLE INC,XBO,MiFID Composite Cross Market Service,[EQUITY]
AAPL.OQ,Apple Ord Shs APPLE ORD,NSM,NASDAQ Stock Exchange Global Select Market,[EQUITY]
AAPL.DG,Apple Ord Shs APPLE ORD,GCD,Cboe EDGX Exchange - Nasdaq Global Select Market,[EQUITY]
AAPL.Z,Apple Ord Shs APPLE ORD APPLE INC,GCZ,Cboe BZX Exchange - Nasdaq Global Select Market,[EQUITY]
AAPL.B,Apple Ord Shs APPLE ORD,GSB,Boston SE when trading NASDAQ Global Select Market,[EQUITY]
AAPLE.MI,Apple Ord Shs APPLE ORD,MIL,Milan Stock Exchange,[EQUITY]
AAPL.CCP,Apple 100 CDR Representing 12 Ord Shs APPLE 100 CDR REP 12 ORD,CAQ,Canadian Composite Quote/Trade,[EQUITY]
AAPL.PH,Apple Ord Shs APPLE ORD APPLE INC,GSX,NASDAQ OMX PHLX Global Select Market,[EQUITY]
AAPL.DY,Apple Ord Shs APPLE ORD,GDA,Cboe EDGA Exchange - Nasdaq Global Select Market,[EQUITY]


#### Example - Vessels
List all Very Large Gas Containers (VLGC) that are in service within the last week.

In [9]:
// Build date range expression for the location of the vessel for the last 7 days
var date7DaysAgo = DateTime.Now.AddDays(-7);
var dateRange = @$"LocationTimestamp ge {date7DaysAgo:yyyy-MM-dd} and LocationTimestamp lt {DateTime.Now:yyyy-MM-dd}";
dateRange

LocationTimestamp ge 2022-01-17 and LocationTimestamp lt 2022-01-24

In [10]:
// Show the location of each vessel and some general details such as location, destination and coordinates.
filter = $"RCSAssetTypeLeaf eq 'VLGC' and VesselStatus eq 'IN SERVICE' and {dateRange}";
select = "RIC, SynonymSearchAll, DTSource, LocationTimestamp, IMO,Latitude, Longitude, LocationDestination";

response = Search.Definition(Search.View.VesselPhysicalAssets).Filter(filter)
                                                              .Select(select)
                                                              .Top(100)
                                                              .GetData();
response.Data.Table

C}KG7309924905,Chinese Korean Japanese Liquefied Petroleum Gas Very Large Gas Carrier,East China Sea|Korea Tanker Zone,23/01/2022 4:43:55 PM,9864306,32.983318328857,127.031814575195,CNLYG [Lianyungang]
C}KY7309491971,Egyptian Liquefied Petroleum Gas Very Large Gas Carrier,Red Sea|Suez Canal Tanker Zone|Egypt,20/01/2022 2:56:07 AM,9350290,28.743871688843,33.145748138428,WADIFEIRAN
C}KY7309529540,Indonesian India Liquefied Petroleum Gas Very Large Gas Carrier,Indonesia|Banten|Java Sea|Singapore Tanker Zone|Sunda Strait Waypoint,22/01/2022 1:50:20 PM,9643348,-5.940100193024,105.963020324707,SEKONG_IDN
C}KY7309586097,Indonesian India Liquefied Petroleum Gas Very Large Gas Carrier,Indonesia|East Java|Java Sea|Lawi-Lawi Terminal Tanker Zone,23/01/2022 12:54:35 AM,9685217,-7.600648403168,113.991188049316,KALBUT SITUBONDO
C}KG7309693092,Chinese Korean Japanese Liquefied Petroleum Gas Very Large Gas Carrier,East China Sea|Korea Tanker Zone,23/01/2022 3:21:03 PM,9703007,38.799369812012,118.180328369141,CN TXG [Tianjin]
C}KG7309708993,Chinese Liquefied Petroleum Gas Very Large Gas Carrier,Guangdong|South China Sea|Canton Tanker Zone|China (Mainland),22/01/2022 11:41:05 AM,9735036,22.50407409668,113.835632324219,CN SHK [Shekou]
C}VS7309843472,India Indonesian Liquefied Petroleum Gas Very Large Gas Carrier,Java Sea|Lawi-Lawi Terminal Tanker Zone,23/01/2022 9:44:47 PM,9780005,-2.873681783676,118.488647460938,ID BPN [Balikpapan]
C}KY7309843662,Indian India Liquefied Petroleum Gas Very Large Gas Carrier Qatari,Arabia Sea|West Coast India Tanker Zone,23/01/2022 4:42:24 PM,9800099,18.270742416382,68.629486083984,QA UMS [Mesaieed]
C}KY7309843818,Indian Liquefied Petroleum Gas Very Large Gas Carrier,Arabia Sea|West Coast India Tanker Zone,23/01/2022 7:05:08 PM,9752292,21.574384689331,67.013847351074,AEPRA
C}KY7309848634,Indian India Liquefied Petroleum Gas Very Large Gas Carrier,Bay of Bengal|Chittagong Tanker Zone,19/01/2022 4:48:23 PM,9810044,20.995178222656,87.914794921875,HALDIA [Haldia]
C}KY7309951010,India Indian Liquefied Petroleum Gas Very Large Gas Carrier,West Indian Ocean|West Coast India Tanker Zone,23/01/2022 6:27:24 PM,9892951,11.744348526001,73.039649963379,RUWAIS UAE [Jebel Dhanna/Ruwais]


#### Example - Floating Rate Notes
Retrieve all floating rate notes (floating coupon) in the healthcare sector which are issued in Euros.  For each, ensure the coupon rate is great than 2 and our Moody's rating is greater than 'Baa1'.

To determine the rating, we apply the following logic utilizing the RatingsScope property - which is a nested property providing additional details:

- RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10)

The RatingType of 'MIS' refers to the "Moody's Long-term Issuer Rating".
The CurrentRatingRank < 10 instructs search to look for ratings less than 10 ('Baa1').  **Note**: The lower the rank, the better the rating.

In [11]:
filter = @"CouponRate gt 2 and
           RatingsScope(RatingType eq 'MIS' and CurrentRatingRank lt 10) and
           RCSCouponTypeName eq 'Floating Coupon' and
           IsActive eq true and
           RCSTRBC2012Name eq 'Healthcare' and
           RCSCurrencyLeaf xeq 'Euro'";
select = "CouponCurrency, IssuerName, IssueDate, RIC, CouponRate, ISIN, Ticker, RCSTRBC2012Leaf, MaturityDate, AssetState";

response = Search.Definition(Search.View.GovCorpInstruments).Filter(filter)
                                                            .Select(select)
                                                            .GetData();
response.Data.Table

EUR,Sanofi FR,01/07/1983 12:00:00 AM,FR0000140022=,15.5,FR0000140022,SASY,Pharmaceuticals (NEC)
EUR,Sanofi FR,01/10/1987 12:00:00 AM,FR0000047656=AURO,15.5,FR0000047656,SASY,Pharmaceuticals (NEC)


#### Example - Outstanding Green Bonds
List the outstanding Green Bonds for Telefonica.

In [12]:
select = @"ssuerName, DocumentTitle, RIC, CouponRate, ISIN, IssuerTicker, DebtTypeDescription, 
           AmountOutstanding, FaceOutstanding";

response = Search.Definition(Search.View.GovCorpInstruments).Filter("IssuerName eq 'Telefonica' and IsGreenBond eq true")
                                                            .Select(select)
                                                            .GetData();
response.Data.Table

"Telefonica Europe BV, Bond, TEF 2.502",ES210981985=,2.502,XS2109819859,TEFBV,Junior Subordinated Note,500000000
"Telefonica Emisiones SAU, Plain Vanilla Fixed Coupon Bond, TEF 1.069 05-Feb-2024",ES194600445=,1.069,XS1946004451,TEFEMI,Senior Note,1000000000


#### Example - Equity Index listing in the U.S.A

In [13]:
// Based on the Asset Category (Equity Index), list the top 10 indexes within the USA.
filter = "RCSIndexCountryGroupLeaf xeq 'United States' and RCSAssetCategoryLeaf xeq 'Equity Index'";

response = Search.Definition(Search.View.IndexInstruments).Filter(filter)
                                                          .Select("DocumentTitle, RIC, RCSTRBC2012Leaf")
                                                          .GetData();
response.Data.Table

"S&P 500 Index - CBOE, Equity Index",.SPX,Magazine Publishing
"Dow Jones Industrial Average Index, Equity Index",.DJI,Consumer Publishing (NEC)
"NASDAQ 100 Index, Equity Index",.NDX,UK Investment Trusts
"NASDAQ Composite Index, Equity Index",.IXIC,Financial & Commodity Market Operators & Service Providers (NEC)
"Refinitiv United States Price Return Index, Equity Index",.TRXFLDUSP,Professional Information Services (NEC)
"Dow Jones Composite Index, Equity Index",.DJA,Consumer Publishing (NEC)
"Dow Jones Transport Average Index, Equity Index",.DJT,Consumer Publishing (NEC)
"S&P 600 Small Cap Index, Equity Index",.SPCY,Magazine Publishing
"Dow Jones Utility Average Index, Equity Index",.DJU,Consumer Publishing (NEC)
"NYSE Composite Index, Equity Index",.NYA,Financial & Commodity Market Operators & Service Providers (NEC)


#### Example - Search for Brazil global sovereign bonds in USD with a duration greater than 1

In [14]:
filter = @"RCSIssuerCountryLeaf eq 'Brazil' and
           RCSCurrencyLeaf eq 'US Dollar' and
           RCSTRBC2012Leaf eq 'Government & Government Finance' and
           OptionAdjustedDuration gt 1";
select = "RIC, ISIN, MaturityDate, CouponRate, IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration";

response = Search.Definition(Search.View.FixedIncomeInstruments).Filter(filter)
                                                                .Select(select)
                                                                .GetData();
response.Data.Table

BR012631375=,US105756AR10,15/04/2024 12:00:00 AM,8.875,SOVERGRN,1029030000,2.022868
BR017244388=,US105756AZ36,15/04/2024 12:00:00 AM,8.875,SOVERGRN,85557000,2.022868
105756BV1=,US105756BV13,07/01/2025 12:00:00 AM,4.25,SOVERGRN,4300000000,2.769654
BR021210641=,US105756BF62,04/02/2025 12:00:00 AM,8.75,SOVERGRN,688201000,2.632758
105756CD0=,US105756CD06,06/06/2025 12:00:00 AM,2.875,SOVERGRN,1750000000,3.175001
105756BX7=,US105756BX78,07/04/2026 12:00:00 AM,6,SOVERGRN,2176245000,3.682704
105756AE0=,US105756AE07,15/05/2027 12:00:00 AM,10.125,SOVERGRN,765023000,4.252306
105756BZ2=,US105756BZ27,13/01/2028 12:00:00 AM,4.625,SOVERGRN,3000000000,5.070227
105756CA6=,US105756CA66,30/05/2029 12:00:00 AM,4.5,SOVERGRN,2000000000,6.063612
BR010862957=,US105756AL40,06/03/2030 12:00:00 AM,12.25,SOVERGRN,237592000,5.677658


In [15]:
// Alternatively, you have the option of trying it this way which will render the same results
filter = @"IssuerCountryName eq 'Brazil' and
           OptionAdjustedDuration gt 1 and
           Currency eq 'USD' and IndustrySectorDescription eq 'Sovereign'";
select = "RIC, ISIN, MaturityDate, CouponRate, IndustrySector, FaceOutstandingUSD, OptionAdjustedDuration";

response = Search.Definition(Search.View.FixedIncomeInstruments).Filter(filter)
                                                                .Select(select)
                                                                .GetData();
response.Data.Table

BR012631375=,US105756AR10,15/04/2024 12:00:00 AM,8.875,SOVERGRN,1029030000,2.022868
BR017244388=,US105756AZ36,15/04/2024 12:00:00 AM,8.875,SOVERGRN,85557000,2.022868
105756BV1=,US105756BV13,07/01/2025 12:00:00 AM,4.25,SOVERGRN,4300000000,2.769654
BR021210641=,US105756BF62,04/02/2025 12:00:00 AM,8.75,SOVERGRN,688201000,2.632758
105756CD0=,US105756CD06,06/06/2025 12:00:00 AM,2.875,SOVERGRN,1750000000,3.175001
105756BX7=,US105756BX78,07/04/2026 12:00:00 AM,6,SOVERGRN,2176245000,3.682704
105756AE0=,US105756AE07,15/05/2027 12:00:00 AM,10.125,SOVERGRN,765023000,4.252306
105756BZ2=,US105756BZ27,13/01/2028 12:00:00 AM,4.625,SOVERGRN,3000000000,5.070227
105756CA6=,US105756CA66,30/05/2029 12:00:00 AM,4.5,SOVERGRN,2000000000,6.063612
BR010862957=,US105756AL40,06/03/2030 12:00:00 AM,12.25,SOVERGRN,237592000,5.677658


#### Example - Top executives

In [16]:
// Using the People View, we can utilize the OfficerDirector property, which contains a number of nested or sub-properties 
// that allow me to perform a more fine-grained search against active people that have been assigned a title code of 
// CEO (Chief Executive Officer). The results are listed based on a default rank defined by the search service.
filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')";
select = @"FullName, PrimaryTitleAsReported, PrimaryOfficerDirector, RIC, 
           OfficerDirectorActiveCompanies, PrimaryOfficerDirectorRIC";
           
response = Search.Definition(Search.View.People).Filter(filter)
                                                .Select(select)
                                                .Top(20)
                                                .GetData();
response.Data.Table

Satya Nadella,"Chairman of the Board, Chief Executive Officer",Microsoft Corp,"[Starbucks Corp, Chandler Insurance Company Ltd, Microsoft Corp]",MSFT.O
Tim D. Cook,"Chief Executive Officer, Director",Apple Inc,"[Apple Inc, Nike Inc, Apple Retail Uk Ltd]",AAPL.O
Andrew R. Jassy,"President, Chief Executive Officer, Director",Amazon.com Inc,"[Coupa Software Inc, Amazon.com Inc]",AMZN.O
Sundar Pichai,"Chief Executive Officer, Director",Alphabet Inc,"[Alphabet Inc, Ruba Inc (Fka Kudo Inc)]",GOOGL.O
Mark Zuckerberg,"Chairman of the Board, Chief Executive Officer, Founder",Meta Platforms Inc,[Meta Platforms Inc],FB.O
Warren E. Buffett,"Chairman of the Board, Chief Executive Officer",Berkshire Hathaway Inc,"[Grinnell College, Bill & Melinda Gates Foundation Trust, Berkshire Hathaway Inc, Kraft Heinz Intermediate Corporation II, Kraft Heinz Foods Co, Berkshire Hathaway Energy Co]",BRKa
Daniel Zhang,"Chairman of the Board, Chief Executive Officer",Alibaba Group Holding Ltd,"[Alibaba Group Holding Ltd, DiDi Global Inc]",BABA.K
Fan Jiang,President - Taobao and Tmall,Alibaba Group Holding Ltd,"[Alibaba Group Holding Ltd, Umeng Technology Ltd]",BABA.K
Huateng Ma,"Executive Chairman of the Board, Chief Executive Officer",Tencent Holdings Ltd,"[Tencent Holdings Ltd, Tencent Technology Beijing Co Ltd]",0700.HK
Al Francis Kelly,"Chairman of the Board, Chief Executive Officer",Visa Inc,[Visa Inc],V
Joaquin Boix Duato,"Chief Executive Officer, Director",Johnson & Johnson,"[Johnson & Johnson, Hess Corp]",JNJ


#### Example - Live Options
The following example will specify an underlying instrument (RIC) and pull the list of options that are active, i.e. not expired.

In [17]:
// Define our underlying
var underlying = "AAPL.O";

// To search for our instrument, I'm utilizing the property: 'UnderlyingQuoteRIC'.  The property does not support an 
// exact match 'xeq', so we must use the partial match 'eq' to find all hits. Depending on the instrument you specify, 
// this may introduce additional hits.  For example, if you are using the underlying 'AAPL.O', search will also find a 
// hit on 'AAPL.OQ', for example. To protect against this, we can utilize a combination of the built-in functions, 
// 'startswith' and 'endwith' to perform our match.
filter = @$"AssetState eq 'AC' and
            startswith(UnderlyingQuoteRIC,'{underlying}') and
            endswith(UnderlyingQuoteRIC,'{underlying}') and
            RCSAssetClass eq 'OPT' and IsChain eq false and
            ExpiryDate ge {DateTime.Now:yyyy-MM-dd}";
            
select = "DocumentTitle, RIC, ExpiryDate, CallPutOption, StrikePrice";
           
response = Search.Definition(Search.View.EquityDerivativeQuotes).Filter(filter)
                                                                .Select(select)
                                                                .Top(10000)
                                                                .GetData();
response.Data.Table

"OPRA Apple Equity Option 180 Call Feb 2022 , Equity Cash Option, Call 180 USD 18-Feb-2022, OPRA",AAPLB182218000.U,18/02/2022 12:00:00 AM,Call,180
"OPRA Apple Equity Option 170 Call Feb 2022 , Equity Cash Option, Call 170 USD 18-Feb-2022, OPRA",AAPLB182217000.U,18/02/2022 12:00:00 AM,Call,170
"OPRA Apple Equity Option 175 Call Feb 2022 , Equity Cash Option, Call 175 USD 18-Feb-2022, OPRA",AAPLB182217500.U,18/02/2022 12:00:00 AM,Call,175
"OPRA Apple Equity Option 185 Call Feb 2022 , Equity Cash Option, Call 185 USD 18-Feb-2022, OPRA",AAPLB182218500.U,18/02/2022 12:00:00 AM,Call,185
"OPRA Apple Equity Option 150 Put Feb 2022 , Equity Cash Option, Put 150 USD 18-Feb-2022, OPRA",AAPLN182215000.U,18/02/2022 12:00:00 AM,Put,150
"OPRA Apple Equity Option 190 Call Feb 2022 , Equity Cash Option, Call 190 USD 18-Feb-2022, OPRA",AAPLB182219000.U,18/02/2022 12:00:00 AM,Call,190
"OPRA Apple Equity Option 180 Call Mar 2022 , Equity Cash Option, Call 180 USD 18-Mar-2022, OPRA",AAPLC182218000.U,18/03/2022 12:00:00 AM,Call,180
"OPRA Apple Equity Option 170 Put Mar 2022 , Equity Cash Option, Put 170 USD 18-Mar-2022, OPRA",AAPLO182217000.U,18/03/2022 12:00:00 AM,Put,170
"OPRA Apple Equity Option 170 Put Jan 2022 , Equity Cash Option, Put 170 USD 28-Jan-2022, OPRA",AAPLM282217000.U,28/01/2022 12:00:00 AM,Put,170
"OPRA Apple Equity Option 180 Call Jan 2022 , Equity Cash Option, Call 180 USD 28-Jan-2022, OPRA",AAPLA282218000.U,28/01/2022 12:00:00 AM,Call,180
"OPRA Apple Equity Option 170 Call Mar 2022 , Equity Cash Option, Call 170 USD 18-Mar-2022, OPRA",AAPLC182217000.U,18/03/2022 12:00:00 AM,Call,170
